In [26]:
import os
import zipfile
import random
import shutil

# Define paths
dataset_dir = "RESIDE-6K"
train_zip_path = os.path.join(dataset_dir, "train.zip")
test_zip_path = os.path.join(dataset_dir, "test.zip")

final_dataset_dir = "RESIDE-6K-OUR"
# Create final dataset directory if it doesn't exist
if not os.path.exists(final_dataset_dir):
    os.makedirs(final_dataset_dir)

train_dir = os.path.join(final_dataset_dir, "train")
test_dir = os.path.join(final_dataset_dir, "test")

train_gt_dir = os.path.join(train_dir, "GT")
train_hazy_dir = os.path.join(train_dir, "hazy")

test_gt_dir = os.path.join(test_dir, "GT")
test_hazy_dir = os.path.join(test_dir, "hazy")

# Create train and test directories if they don't exist
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    os.makedirs(train_gt_dir)
    os.makedirs(train_hazy_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    os.makedirs(test_gt_dir)
    os.makedirs(test_hazy_dir)

In [29]:
# Extract images from train.zip and test.zip
def extract_images(zip_path, output_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    return os.listdir(output_dir)

# Extract images train
extracted_train = extract_images(train_zip_path, dataset_dir)
extracted_train_path = dataset_dir + '/train'
print(extracted_train_path)
print("Number of images in train dataset:")
print(f"GT: {len(os.listdir(extracted_train_path+'/GT'))}")
print(f"HAZY: {len(os.listdir(extracted_train_path+'/hazy'))}")
print("-"*100)

# Extract images test
extracted_test = extract_images(test_zip_path, dataset_dir)
extracted_test_path = dataset_dir + '/test'
print(extracted_test_path)
print("Number of images in test dataset:")
print(f"GT: {len(os.listdir(extracted_test_path+'/GT'))}")
print(f"HAZY: {len(os.listdir(extracted_test_path+'/hazy'))}")

RESIDE-6K/train
Number of images in train dataset:
GT: 6000
HAZY: 6000
----------------------------------------------------------------------------------------------------
RESIDE-6K/test
Number of images in test dataset:
GT: 1000
HAZY: 1000


In [30]:
# Pick 5 random images from extracted images
def pick_random_images(extracted_dir, num_images):
    images = os.listdir(extracted_dir)
    return random.sample(images, num_images)

# extracted_train_dir = os.path.join(final_dataset_dir, "train")
# selected_train_images = pick_random_images(extracted_train_dir, 5)
# extracted_test_dir = os.path.join(final_dataset_dir, "test")
selected_train_images_GT = pick_random_images(extracted_train_path+'/GT', 100)
selected_train_images_HAZY = pick_random_images(extracted_train_path+'/hazy', 100)
selected_test_images_GT = pick_random_images(extracted_test_path+'/GT', 100)
selected_test_images_HAZY = pick_random_images(extracted_test_path+'/hazy', 100)

print(f"Selected Test: {len(selected_train_images_GT), len(selected_train_images_HAZY)}")
print(f"Selected Train: {len(selected_test_images_GT), len(selected_test_images_HAZY)}")

# Move selected images to train_gt_dir
for image in selected_train_images_GT:
    shutil.move(os.path.join(extracted_train_path, 'GT', image), os.path.join(train_gt_dir, image))

# Move selected images to train_hazy_dir
for image in selected_train_images_HAZY:
    shutil.move(os.path.join(extracted_train_path, 'hazy', image), os.path.join(train_hazy_dir, image))

# Move selected images to test_gt_dir
for image in selected_test_images_GT:
    shutil.move(os.path.join(extracted_test_path, 'GT', image), os.path.join(test_gt_dir, image))

# Move selected images to test_hazy_dir
for image in selected_test_images_HAZY:
    shutil.move(os.path.join(extracted_test_path, 'hazy', image), os.path.join(test_hazy_dir, image))

print("Final dataset created successfully!")

Selected Test: (100, 100)
Selected Train: (100, 100)
Final dataset created successfully!


In [1]:
import torch
import numpy as np
from PIL import Image

img_path = "dummy_data/RESIDE-OUT/train/GT/0001.png"

img = Image.open(img_path)

img_tensor = torch.tensor(np.array(img))

print("Shape of the img passed to model:", img_tensor.shape)


Shape of the img passed to model: torch.Size([413, 550, 3])


In [9]:
from datasets.loader import PairLoader, SingleLoader
from torch.utils.data import DataLoader
import json

setting_filename = "configs/outdoor/dehazeformer-t.json" #os.path.join('configs', args.exp, args.model+'.json')

dataset_dir = "data/RESIDE-OUT" #os.path.join(args.data_dir, args.dataset)
with open(setting_filename, 'r') as f:
    setting = json.load(f)

train_dataset = PairLoader(dataset_dir, 'train', 'train', 
								setting['patch_size'], setting['edge_decay'], setting['only_h_flip'])
train_loader = DataLoader(train_dataset,
                              batch_size=setting['batch_size'],
                              shuffle=True,
                              num_workers=2,
                              pin_memory=True,
                              drop_last=True)

val_dataset = PairLoader(dataset_dir, 'test', setting['valid_mode'], 
							  setting['patch_size'])
val_loader = DataLoader(val_dataset,
                        batch_size=setting['batch_size'],
                        num_workers=2,
                        pin_memory=True)

print(f'Len train: {len(train_loader)}')
print(f'Len val: {len(train_loader)}')

for batch in train_loader:
    source_img = batch['source']
    print(f'Hazy shape: {source_img.shape}')
    target_img = batch['target']
    print(f'target shape: {source_img.shape}')
    break

Len train: 48
Len val: 48
Hazy shape: torch.Size([8, 3, 64, 64])
target shape: torch.Size([8, 3, 64, 64])
